# Robustness Analysis of Random Forest Classifiers Against Adversarial Attacks Using TTTS

### Description:

##### This notebook leverages the TTTS (Tree Test Time Simulation) package to systematically assess the robustness of various Random Forest classifiers against adversarial attacks. The analysis is conducted across an extensive collection of 50 datasets, ensuring a broad and deep understanding of classifier performance under adversarial conditions. Key components of the notebook include:

#### <ins>Dataset Preparation and Organization:</ins>

##### - Initialization of base_path to set the directory for dataset storage.
##### - Curation of dataset_list, bigger_datasets_list, and more_bigger_datasets_list to segregate datasets based on size for differentiated processing.

#### <ins>Adversarial Robustness Assessment of Random Forest Classifiers:</ins>

##### - Implementation of a robust testing framework using 5-fold StratifiedKFold cross-validation.
##### - Configuration of various Random Forest classifiers, each with unique probability adjustment strategies.
##### - Functionality for training classifiers, generating adversarial examples, applying defensive techniques, and evaluating multiple performance metrics.
##### - Parallel compilation of evaluations in results_df_dtattack DataFrame, offering a structured analysis of each classifier's resilience against adversarial attacks.

#### <ins>Evaluation Against Zoo Adversarial Attacks:</ins>

##### - Detailed robustness analysis of Decision Tree classifiers, inclusive of a custom MonteCarlo RF implementation, when faced with Zoo adversarial attacks.
##### - Parallel processing to efficiently train classifiers, generate adversarial samples, apply optional defenses, and compute key performance metrics.
##### -Aggregation and visualization of results, providing insights into the defensive capabilities of each classifier against Zoo attacks.

In [6]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, log_loss, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer,fetch_lfw_pairs,load_digits,load_iris,load_wine
from sklearn.tree import _tree
from sklearn.utils.validation import check_is_fitted
from sklearn.model_selection import StratifiedKFold
from art.defences.preprocessor import FeatureSqueezing,GaussianAugmentation

from art.attacks.evasion import FastGradientMethod,AutoProjectedGradientDescent,ThresholdAttack
from art.attacks.evasion import ZooAttack,HopSkipJump, BoundaryAttack, DecisionTreeAttack
from art.attacks.evasion import HighConfidenceLowUncertainty, ProjectedGradientDescent
from sklearn.utils import Bunch
from joblib import Parallel, delayed

from art.estimators.classification import SklearnClassifier
from TTTS import MonteCarloRandomForestClassifier

## Parallel Loading and Processing of Multiple Datasets

In [7]:
# Define the base path for file storage and lists of dataset filenames
# 'base_path' represents the directory where the dataset files are stored.
# 'dataset_list' contains filenames of datasets to be processed.
# 'bigger_datasets_list' and 'more_bigger_datasets_list' are subsets of larger datasets for additional processing options.

# Base path where the files are stored
base_path = "../data/"

# List of files
dataset_list = [
    "!ar4.csv",
    "!bodyfat.csv",
    "Kaggle_Surgical-deepnet.csv",
    "MaternalBinary.csv",
    "OPENML_philippine.csv",
    "AcousticExtinguisherFire.csv",
    "acute-inflammation.csv",
    "acute-nephritis.csv",
    "AP_Colon_Lung.csv",
    "backache.csv",
    "blood.csv",
    "chess-krvkp.csv",
    "cloud.csv",
    "congressional-voting.csv",
    "credit-approval.csv",
    "dresses-salesN.csv",
    "echocardiogram.csv",
    "haberman-survival.csv",
    "heart_failure_clinical_records_dataset.csv",
    "heart-hungarian.csv",
    "hill-valley.csv",
    "horse-colic.csv",
    "ilpd-indian-liver.csv",
    "no2.csv",
    "kaggle_REWEMA.csv",
    "lowbwt.csv",
    "madelon.csv",
    "Mesothelioma.csv",
    "MIMIC2.csv",
    "molec-biol-promoter.csv",
    "oil_spill.csv",
    "oocytes_merluccius_nucleus_4d.csv",
    "oocytes_trisopterus_nucleus_2f.csv",
    "ozone.csv",
    "Parkinson_Multiple_Sound_Recording.csv",
    "PC1 Software defect prediction.csv",
    "pd_speech_features.csv",
    "pima.csv",
    "Pistachio_28_Features_Dataset.csv",
    "plasma_retinol.csv",
    "primary-tumorNumeric.csv",
    "seismic-bumps.csv",
    "sleuth_case2002.csv",
    "spambase.csv",
    "spect.csv",
    "spectf.csv",
    "statlog-australian-credit.csv",
    "statlog-heart_.csv",
    "ThoraricSurgery.csv",
    "triazines.csv",
]

# Function to load dataset
def load_dataset(file_name, path):
    try:
        data = pd.read_csv(path + file_name)
        # Use all columns except the last one as features
        X = data.iloc[:, :-1]
        # Use the last column as the target class
        y = data.iloc[:, -1]
        return (file_name, Bunch(data=X, target=y))
    except Exception as e:
        print(f"Error loading {file_name}: {e}")
        return None


# Parallel loading of datasets (using all available cores with n_jobs=-1)
datasets = Parallel(n_jobs=-1)(
    delayed(load_dataset)(file_name, base_path) for file_name in dataset_list
)

# Filter out None values in case of loading errors
datasets = [dataset for dataset in datasets if dataset is not None]

# Now 'datasets' is a list of tuples, where each tuple contains file_name and the corresponding dataset as a Bunch object.


## Adversarial Robustness Assessment of Random Forest Classifiers

In [ ]:
# This notebook cell conducts an adversarial robustness assessment of various Random Forest classifiers using 5-fold cross-validation. The process involves:

# Initializing StratifiedKFold for equitable distribution across folds.
# Defining a suite of Decision Tree classifiers, each tailored with unique probability adjustment strategies.
# Implementing the evaluate_classifier function to train classifiers, generate adversarial examples, apply defensive techniques (if applicable), 
# and evaluate performance metrics such as AUC, F1 score, Log Loss, Accuracy, and Runtime.
# Iterating over datasets and classifiers, the cell conducts cross-validation, compiles the evaluations in parallel, 
# and organizes the results in a DataFrame results_df_dtattack for a structured analysis of each classifier's resilience against adversarial attacks.

# Initialize a 5-fold StratifiedKFold object for cross-validation, ensuring each fold is a good representative of the whole.
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize a list to store the evaluation results of the classifiers.
results = []

# Define a dictionary of classifiers to evaluate. Each key is the classifier name and the value is the classifier object.
classifiers = {
    "RandomForest": RandomForestClassifier(random_state=123, n_estimators=10),
    "MonteCarloRandomForest_Fix_Prob": MonteCarloRandomForestClassifier(
        random_state=123, prob_type="fixed", n_estimators=10
    ),
    "MonteCarloRandomForest_Depth_Prob": MonteCarloRandomForestClassifier(
        random_state=123, prob_type="depth", n_estimators=10
    ),
    "MonteCarloRandomForest_Agreement_Prob": MonteCarloRandomForestClassifier(
        random_state=123, prob_type="agreement", n_estimators=10
    ),
    "MonteCarloRandomForest_Bayes_Prob": MonteCarloRandomForestClassifier(
        random_state=123, prob_type="bayes", n_estimators=10
    ),
    "MonteCarloRandomForest_Confidence_Prob": MonteCarloRandomForestClassifier(
        random_state=123, prob_type="confidence", n_estimators=10
    ),
    "MonteCarloRandomForest_Distance_Prob": MonteCarloRandomForestClassifier(
        random_state=123, prob_type="distance", n_estimators=10
    ),
    "FeatureSqueezing": RandomForestClassifier(random_state=123, n_estimators=10),
}


# Define a function to evaluate a classifier. This function trains the classifier, makes predictions, and calculates evaluation metrics.
def evaluate_classifier(dataset_name, dataset, clf_name, clf, train_index, test_index):
    # Prepare the data by filling NA values with 0 and extracting features (X) and target variable (y).
    X, y = dataset.data.fillna(0).values, dataset.target
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the classifier and measure the time taken for training.
    start_time = time.time()
    clf.fit(X_train, y_train)

    # Make predictions on the test set.
    pred_probs = clf.predict_proba(X_test)
    preds = clf.predict(X_test)
    runtime = time.time() - start_time

    # Calculate evaluation metrics: AUC, F1-score, Log Loss, Accuracy.
    auc = roc_auc_score(y_test, pred_probs[:, 1], multi_class="ovr")
    f1 = f1_score(y_test, preds, average="macro")
    logloss = log_loss(y_test, pred_probs)
    accuracy = accuracy_score(y_test, preds)

    # Return all calculated metrics.
    return [dataset_name, clf_name, auc, f1, logloss, accuracy, runtime]


# Prepare tasks for parallel execution. This involves creating a list of delayed task objects.
all_tasks = []
for dataset_name, dataset in datasets:
    print(dataset_name)
    X, y = dataset.data.fillna(0).values, dataset.target

    # Create a task for each fold of each classifier for each dataset.
    for clf_name, clf in classifiers.items():
        for i, (train_index, test_index) in enumerate(skf.split(X, y)):
            task = delayed(evaluate_classifier)(
                dataset_name, dataset, clf_name, clf, train_index, test_index
            )
            all_tasks.append(task)

# Execute all tasks in parallel using joblib's Parallel. n_jobs=-1 means using all available CPU cores.
results = Parallel(n_jobs=-1)(all_tasks)

# Convert the list of results into a DataFrame for easier analysis and visualization. Each row contains the performance metrics of a classifier on a dataset.
results_df_wo_rf = pd.DataFrame(
    results,
    columns=["Dataset", "Classifier", "AUC", "F1", "LogLoss", "Accuracy", "Runtime"],
)


!ar4.csv
!bodyfat.csv
Kaggle_Surgical-deepnet.csv
MaternalBinary.csv
OPENML_philippine.csv
AcousticExtinguisherFire.csv
acute-inflammation.csv
acute-nephritis.csv
backache.csv
blood.csv
chess-krvkp.csv
cloud.csv
congressional-voting.csv
credit-approval.csv
dresses-salesN.csv
echocardiogram.csv
haberman-survival.csv
heart_failure_clinical_records_dataset.csv
heart-hungarian.csv
hill-valley.csv
horse-colic.csv
ilpd-indian-liver.csv
no2.csv
kaggle_REWEMA.csv
lowbwt.csv
madelon.csv
Mesothelioma.csv
MIMIC2.csv
molec-biol-promoter.csv
oil_spill.csv
oocytes_merluccius_nucleus_4d.csv
oocytes_trisopterus_nucleus_2f.csv
ozone.csv
Parkinson_Multiple_Sound_Recording.csv
PC1 Software defect prediction.csv
pd_speech_features.csv
pima.csv
Pistachio_28_Features_Dataset.csv
plasma_retinol.csv
primary-tumorNumeric.csv
seismic-bumps.csv
sleuth_case2002.csv
spambase.csv
spect.csv
spectf.csv
statlog-australian-credit.csv
statlog-heart_.csv
ThoraricSurgery.csv
triazines.csv


In [ ]:
# This cell aggregates the performance metrics for each classifier and dataset combination from results_df_dtattack, 
# calculating the mean and standard deviation of AUC, F1 score, Log Loss, Accuracy, and Runtime.

summary_wo = results_df_wo_rf.groupby(['Dataset','Classifier']).agg({
    'AUC': ['mean', 'std'],
    'F1': ['mean', 'std'],
    'LogLoss': ['mean', 'std'],
    'Accuracy': ['mean', 'std'],
    'Runtime': ['mean', 'std']
}).reset_index()

print(summary_wo)

## Performance Evaluation of RF Classifiers and Monte Carlo RF Classifier under Zoo Attacks

In [ ]:
# This cell evaluates the robustness of RF classifiers, including our MonteCarlo RF implementation, against Zoo adversarial attacks using 5-fold cross-validation. 
# It trains classifiers, generates adversarial samples, optionally applies defenses, and measures performance metrics. 
# The evaluation is executed in parallel for efficiency, and results are aggregated and displayed, providing insights into each classifier's defense against Zoo attacks.

# 5-fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# List to store evaluation results
results = []

# Define classifiers
classifiers = {
    'RandomForest_ZooAttack':  RandomForestClassifier(random_state=123,n_estimators=10),
    'MonteCarloRandomForest_Fix_Prob_ZooAttack':  MonteCarloRandomForestClassifier(random_state=123,prob_type='fixed',n_estimators=10),
    'MonteCarloRandomForest_Depth_Prob_ZooAttack':  MonteCarloRandomForestClassifier(random_state=123,prob_type='depth',n_estimators=10),
    'MonteCarloRandomForest_Agreement_Prob_ZooAttack':  MonteCarloRandomForestClassifier(random_state=123,prob_type='agreement',n_estimators=10),
    'MonteCarloRandomForest_Confidence_Prob_ZooAttack':  MonteCarloRandomForestClassifier(random_state=123,prob_type='confidence',n_estimators=10),
    'MonteCarloRandomForest_Distance_Prob_ZooAttack':  MonteCarloRandomForestClassifier(random_state=123,prob_type='distance',n_estimators=10),
    'FeatureSqueezing_ZooAttack': RandomForestClassifier(random_state=123,n_estimators=10),
}

def evaluate_classifier(dataset_name, dataset, clf_name, clf, train_index, test_index):
    X, y = dataset.data.fillna(0).values, dataset.target
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train classifier
    clf.fit(X_train, y_train)

    if clf_name != 'RandomForest_ZooAttack':
        dummy_clf = RandomForestClassifier(random_state=123,n_estimators=10)
        dummy_clf.fit(X_train, y_train)
        classifier = SklearnClassifier(model=dummy_clf, use_logits=True)
    else:
        classifier = SklearnClassifier(model=clf, use_logits=True)
        
    attack = ZooAttack(classifier=classifier, confidence=0.0, targeted=False, learning_rate=1e-1, max_iter=5,
           binary_search_steps=5, initial_const=1e-3, abort_early=True, use_resize=False, 
           use_importance=False, nb_parallel=2, batch_size=1, variable_h=0.2) 
    x_test_adv = attack.generate(x=X_test)
    
    if clf_name == 'FeatureSqueezing_ZooAttack':
        # Initialize the feature squeezing defence
        defence = FeatureSqueezing(clip_values=(X_train.min(), X_train.max()), bit_depth=4)

        # Fit the defence with training data
        defence.fit(X_train)

        # Apply the defence on testing data
        x_test_adv = defence(x_test_adv)[0]
        
    if clf_name == 'GaussianAugmentation_ZooAttack':
        # Initialize the Gaussian Augmentation defence
        defence = GaussianAugmentation(sigma=1.0)

        # Apply the Gaussian Augmentation on training data
        X_train_augmented, y_train_augmented = defence(X_train, y_train)

        # Retrain the model on the augmented data
        clf.fit(X_train_augmented, y_train_augmented)
    
    start_time = time.time()
    pred_probs = clf.predict_proba(x_test_adv)
    preds = clf.predict(x_test_adv)
    runtime = time.time() - start_time

    # Evaluate
    auc = roc_auc_score(y_test, pred_probs[:, 1], multi_class='ovr')
    f1 = f1_score(y_test, preds, average='macro')
    logloss = log_loss(y_test, pred_probs)
    accuracy = accuracy_score(y_test, preds)

    return [dataset_name, clf_name, auc, f1, logloss, accuracy, runtime]

# Loop through datasets
all_tasks = []

for dataset_name, dataset in datasets:
    print(dataset_name)
    X, y = dataset.data.fillna(0).values, dataset.target
    try:
        # Perform 5-fold cross-validation
        for clf_name, clf in classifiers.items():
            for i, (train_index, test_index) in enumerate(skf.split(X, y)):
                task = delayed(evaluate_classifier)(dataset_name, dataset, clf_name, clf, train_index, test_index)
                all_tasks.append(task)
    except Exception as e:
        print(f"Error in {clf_name} on {dataset_name}: {e}")

# Execute all tasks in parallel
try:
    results = Parallel(n_jobs=-1)(all_tasks)
except Exception as e:
    print(f"Error in {clf_name} on {dataset_name}: {e}")

# Convert results to DataFrame for easy visualization
results_df_ZooAttack = pd.DataFrame(results, columns=['Dataset', 'Classifier', 'AUC', 'F1', 'LogLoss', 'Accuracy', 'Runtime'])

In [ ]:
summary_ZooAttack = results_df_ZooAttack.groupby(['Dataset','Classifier']).agg({
    'AUC': ['mean', 'std'],
    'F1': ['mean', 'std'],
    'LogLoss': ['mean', 'std'],
    'Accuracy': ['mean', 'std'],
    'Runtime': ['mean', 'std']
}).reset_index()

print(summary_ZooAttack)

In [ ]:
pd.concat([summary_wo, summary_ZooAttack]).sort_values(by='Dataset').to_csv('50ds_rf_summary.csv',index=False)

### The End